In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MINST_data', one_hot = True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MINST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MINST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MINST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MINST_data/t10k-labels-idx1-ubyte.gz


In [2]:
def compute_accuracy(test_data, test_label):
    global prediction
    y_pre = sess.run(prediction, feed_dict = {xs: test_data})
    correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.arg_max(ys, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict = {xs: test_data, ys: test_label})
    return result

In [3]:
def Weight(shape):
    weight_mat = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(weight_mat)
def Bias(shape):
    Bias_mat = tf.constant(0.1, shape = shape)
    return tf.Variable(Bias_mat)

In [ ]:
xs = tf.placeholder(tf.float32, [None, 784])
ys = tf.placeholder(tf.float32, [None, 10])
x_image = tf.reshape(xs, [-1, 28, 28, 1])

In [ ]:
weight1 = Weight([5, 5, 1, 32])
Bias1 = Bias([32])
con1 = tf.nn.conv2d(xs, Weight1, strides = [1, 1, 1, 1], padding = "SAME") + Bias1